# Data Cleaning Task - Customer Dataset

This notebook shows the full step-by-step cleaning process for the raw dataset (`raw_customer_data.csv`).
We identify issues (missing values, duplicates, inconsistent formats) and clean them to produce `cleaned_customer_data.csv`.

In [2]:
import pandas as pd
import numpy as np

# Load raw dataset
raw_df = pd.read_csv('raw_customer_data.csv')
print('Raw dataset shape:', raw_df.shape)
raw_df.head()

Raw dataset shape: (10, 7)


,Customer ID,Name,Gender,Country,Date Joined,Age,Email
0,1,Alice,F,USA,2021-01-10,25.0,alice@example.com
1,2,Bob,M,USA,2021-03-15,30.0,bob@example.com
2,2,Bob,M,USA,2021-03-15,30.0,bob@example.com
3,4,Charlie,M,UK,15-04-2021,28.0,charlie@example.com
4,5,David,Male,UK,2021/05/20,NaN,david@example.com


## Step 1: Inspect dataset info

In [4]:
raw_df.info()
raw_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Customer ID  10 non-null     int64  
 1   Name         9 non-null      object 
 2   Gender       9 non-null      object 
 3   Country      9 non-null      object 
 4   Date Joined  9 non-null      object 
 5   Age          9 non-null      float64
 6   Email        9 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 692.0+ bytes


Customer ID    0
Name           1
Gender         1
Country        1
Date Joined    1
Age            1
Email          1
dtype: int64

## Step 2: Remove duplicate rows

In [6]:
before = raw_df.shape[0]
raw_df = raw_df.drop_duplicates()
after = raw_df.shape[0]
print(f'Removed {before-after} duplicate rows.')

Removed 1 duplicate rows.


## Step 3: Handle missing values
- Fill numeric columns with median
- Fill categorical columns with 'unknown'

In [8]:
num_cols = raw_df.select_dtypes(include=['int64','float64']).columns
cat_cols = raw_df.select_dtypes(include=['object']).columns

for c in num_cols:
    raw_df[c] = raw_df[c].fillna(raw_df[c].median())
for c in cat_cols:
    raw_df[c] = raw_df[c].fillna('unknown').str.strip().str.lower()

raw_df.isnull().sum()

Customer ID    0
Name           0
Gender         0
Country        0
Date Joined    0
Age            0
Email          0
dtype: int64

## Step 4: Convert date columns to datetime (if present)

In [10]:
for col in raw_df.columns:
    if 'date' in col.lower():
        raw_df[col] = pd.to_datetime(raw_df[col], errors='coerce')

raw_df.head()

,Customer ID,Name,Gender,Country,Date Joined,Age,Email
0,1,alice,f,usa,2021-01-10,25.0,alice@example.com
1,2,bob,m,usa,2021-03-15,30.0,bob@example.com
3,4,charlie,m,uk,NaT,28.0,charlie@example.com
4,5,david,male,uk,NaT,28.5,david@example.com
5,6,unknown,f,india,2021-06-25,35.0,eve@example.com


## Step 5: Save cleaned dataset

In [12]:
raw_df.to_csv('cleaned_customer_data.csv', index=False)
print('Cleaned dataset saved as cleaned_customer_data.csv')

Cleaned dataset saved as cleaned_customer_data.csv


## Step 6: Create a summary report

In [14]:
summary = {
    'final_rows': raw_df.shape[0],
    'final_columns': raw_df.shape[1],
    'missing_values': int(raw_df.isnull().sum().sum()),
    'duplicates': int(raw_df.duplicated().sum()),
    'numeric_columns_filled': list(num_cols),
    'categorical_columns_filled': list(cat_cols)
}

import json
with open('cleaning_summary.json','w') as f:
    json.dump(summary, f, indent=2)
summary

{'final_rows': 9,
 'final_columns': 7,
 'missing_values': 3,
 'duplicates': 0,
 'numeric_columns_filled': ['Customer ID', 'Age'],
 'categorical_columns_filled': ['Name',
  'Gender',
  'Country',
  'Date Joined',
  'Email']}